In [ ]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
import pandas as pd

try:
    # Conexión a MongoDB
    print("Intentando conectar a MongoDB...")
    client = MongoClient('localhost', 27017)  # Cambia los parámetros si es necesario
    print("Conexión a MongoDB exitosa.")

    # Selecciona la base de datos y la colección
    db = client['CHIRPS']
    collection = db['candelillas de la mar']

    print(client['EVC-SOMMO'].list_collection_names())  # Lista las colecciones de la base de datos


    # Consulta para obtener todos los documentos de la colección
    print("Obteniendo documentos de la colección...")
    data = collection.find()  # Esto devuelve un cursor iterable

    # Convertir el cursor a una lista
    data_list = list(data)

    if not data_list:
        print("No se encontraron documentos.")
    else:
        # Crear listas para almacenar los datos extraídos
        points = []
        timestamps = []

        # Recorrer los documentos y extraer los datos relevantes
        for document in data_list:
            timestamp = document.get('timestamp')  # Extrae el timestamp
            points_data = document.get('point_data', [])  # Extrae el array de puntos

            for point in points_data:
                point_dict = {
                    'point_id': point.get('point_id'),
                    'latitude': point.get('latitude'),
                    'longitude': point.get('longitude'),
                    'value': point.get('value'),
                    'timestamp': timestamp  # Incluir timestamp para cada punto
                }
                points.append(point_dict)

        # Convertir los puntos a un DataFrame
        print(f"Se encontraron {len(points)} puntos. Convirtiendo a DataFrame...")
        df = pd.DataFrame(points)

        # Si la colección tiene un campo '_id', lo eliminamos para evitar problemas
        if '_id' in df.columns:
            df = df.drop(columns=['_id'])

        # Mostrar las primeras filas del DataFrame
        print("DataFrame creado con éxito:")
        display(df)

except ConnectionFailure as e:
    print(f"Error al conectar con MongoDB: {e}")
except OperationFailure as e:
    print(f"Error en la operación de MongoDB: {e}")
except Exception as e:
    print(f"Error inesperado: {e}")


Intentando conectar a MongoDB...
Conexión a MongoDB exitosa.
[]
Obteniendo documentos de la colección...
Se encontraron 60996 puntos. Convirtiendo a DataFrame...
DataFrame creado con éxito:


,point_id,latitude,longitude,value,timestamp
5,point_6,1.774998,-78.725006,0.000000,2009-02-01
9,point_10,1.824997,-78.725006,0.000000,2009-02-01
11,point_12,1.824997,-78.775000,0.000000,2009-02-01
17,point_6,1.774998,-78.725006,0.000000,2009-02-02
21,point_10,1.824997,-78.725006,12.457714,2009-02-02


In [7]:
import pandas as pd
import folium

# Supongamos que 'df' es el DataFrame que tienes
# Extraer las latitudes y longitudes únicas
sensor_locations = []

for row in dfz:
    for sensor in row:  # 'row' es una lista de diccionarios
        location = (sensor['latitude'], sensor['longitude'])
        if location not in sensor_locations:
            sensor_locations.append(location)

# Crear un DataFrame con las ubicaciones únicas
locations_df = pd.DataFrame(sensor_locations, columns=['latitude', 'longitude'])
print(locations_df)


   latitude  longitude
0  1.774998 -78.725006
1  1.824997 -78.725006
2  1.824997 -78.775000


## 2. Crear el mapa con Folium

In [8]:
import folium

# Crear el mapa centrado en la ubicación promedio
map_center = [locations_df['latitude'].mean(), locations_df['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=6)

# Agregar marcadores al mapa
for _, row in locations_df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                popup=f"Lat: {row['latitude']}, Lon: {row['longitude']}").add_to(m)

# Mostrar el mapa (en un entorno compatible como Jupyter Notebook)
m


## 3. (Opcional) Usar Plotly para un mapa interactivo

In [9]:
import plotly.express as px

# Crear un scatter plot con Plotly
fig = px.scatter_geo(locations_df,
                     lat='latitude',
                     lon='longitude',
                     title='Ubicación de Sensores',
                     labels={'latitude': 'Latitud', 'longitude': 'Longitud'})

fig.update_geos(projection_type="natural earth")
fig.show()
